# StyleForge - Real-Time Neural Style Transfer with CUDA Kernels

This notebook demonstrates the StyleForge system with optimized CUDA kernels for real-time neural style transfer.

## Features

- **Optimized Fused Conv+IN+ReLU**: 5-8x faster with shared memory tiling and vectorized loads
- **Fused Instance Norm**: 2-4x faster normalization for style transfer
- **Fused Multi-Head Attention**: Vectorized memory access for ViT models
- **Proper Benchmarking**: CUDA event-based timing with validation

## Requirements

- CUDA 11.0+ GPU with Compute Capability 7.0+
- PyTorch 1.10+ with CUDA support

## 0. Clone Repository and Install Dependencies

Run this cell first to set up the environment.

In [ ]:
# Clone the repository (skip if already cloned)
import os
import subprocess

REPO_URL = "https://github.com/oleeveeuh/StyleForge.git"
REPO_DIR = "/content/StyleForge"  # For Google Colab

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("📌 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("📌 Not running in Google Colab")

# Clone repository if not exists
if IN_COLAB and not os.path.exists(REPO_DIR):
    print(f"Cloning StyleForge repository to {REPO_DIR}...")
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}
elif os.path.exists("StyleForge"):
    %cd StyleForge
    print("Already in StyleForge directory")
elif os.path.exists("../StyleForge"):
    %cd ../StyleForge
    print("Changed to parent StyleForge directory")
else:
    print("Assuming we're in the StyleForge directory")

print("\nRepository setup complete!")

## 1. Install Dependencies and Build Tools

In [ ]:
# Install PyTorch with CUDA support and build tools
import sys
import subprocess

def install_package(package):
    """Install a package with pip."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("=" * 70)
print("STEP 1: Installing Dependencies")
print("=" * 70)

# Check for ninja
print("\nChecking for ninja...")
try:
    result = subprocess.run(['ninja', '--version'], capture_output=True, timeout=5)
    if result.returncode == 0:
        print(f"✓ ninja already installed")
    else:
        raise FileNotFoundError
except (FileNotFoundError, subprocess.TimeoutExpired):
    install_package("ninja")
    print("✓ ninja installed")

# Check PyTorch
print("\nChecking PyTorch...")
try:
    import torch
    print(f"✓ PyTorch {torch.__version__} installed")
except ImportError:
    install_package("torch")

print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## 2. Environment Setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import sys
from pathlib import Path

print("=" * 70)
print("STEP 2: Setting Up Environment")
print("=" * 70)

# Setup path - ensure StyleForge root is in sys.path
styleforge_root = Path.cwd()
if not (styleforge_root / "kernels" / "__init__.py").exists():
    # We might be in notebooks/ subdir
    if (styleforge_root.parent / "kernels" / "__init__.py").exists():
        styleforge_root = styleforge_root.parent
    else:
        # Search upward
        for p in [styleforge_root] + list(styleforge_root.parents):
            if (p / "kernels" / "__init__.py").exists():
                styleforge_root = p
                break

# Add to path if not already there
root_str = str(styleforge_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)
    print(f"Added to path: {root_str}")

if IN_COLAB:
    if REPO_DIR not in sys.path:
        sys.path.insert(0, REPO_DIR)

print(f"Working directory: {Path.cwd()}")
print(f"StyleForge root: {styleforge_root}")
print(f"Device: {device}")

## 3. Import StyleForge Kernels

The kernels will be JIT-compiled on first use. This may take 30-60 seconds.

### Available Kernels:

| Kernel | Purpose | Optimization | Expected Speedup |
|--------|---------|--------------|------------------|
| **FusedInstanceNorm2d** | Fused normalization | Warp reductions, single kernel | 2-4x |
| **FusedConvInstanceNormReLU** | Conv+IN+ReLU fused | Shared memory tiling, float4 vectorization | 5-8x |
| **FusedAttentionV3** | Multi-head attention | Vectorized memory access | 4-8x |

In [ ]:
if torch.cuda.is_available():
    print("=" * 70)
    print("Loading CUDA Kernels...")
    print("=" * 70)
    
    KERNELS_AVAILABLE = False
    
    # Import available kernels
    try:
        from kernels import FusedInstanceNorm2d
        print("✅ FusedInstanceNorm2d imported")
    except ImportError as e:
        print(f"⚠️ FusedInstanceNorm2d not available: {e}")
        FusedInstanceNorm2d = None
    
    try:
        from kernels import FusedAttentionV3
        print("✅ FusedAttentionV3 imported")
    except ImportError as e:
        print(f"⚠️ FusedAttentionV3 not available: {e}")
        FusedAttentionV3 = None
    
    try:
        from kernels import FusedConvInstanceNormReLU
        print("✅ FusedConvInstanceNormReLU imported")
    except ImportError as e:
        print(f"⚠️ FusedConvInstanceNormReLU not available: {e}")
        FusedConvInstanceNormReLU = None
    
    # Check if any kernels loaded
    KERNELS_AVAILABLE = any([FusedInstanceNorm2d is not None, 
                              FusedAttentionV3 is not None,
                              FusedConvInstanceNormReLU is not None])
    
    if KERNELS_AVAILABLE:
        print("\n✅ CUDA kernels loaded successfully!")
    else:
        print("\n⚠️ No CUDA kernels available")

else:
    print("⚠️ CUDA not available")
    KERNELS_AVAILABLE = False
    FusedInstanceNorm2d = None
    FusedAttentionV3 = None
    FusedConvInstanceNormReLU = None

## 4. Fast Style Transfer (Johnson et al.)

This section demonstrates **Fast Neural Style Transfer** using pre-trained weights.

### Available Styles: candy, starry, mosaic, udnie, wave

In [ ]:
if torch.cuda.is_available():
    print("=" * 70)
    print("Fast Style Transfer Setup")
    print("=" * 70)
    
    from models.transformer_net import TransformerNet, AVAILABLE_STYLES
    from pathlib import Path
    
    print(f"Available styles: {', '.join(AVAILABLE_STYLES)}")
    
    # Check for pretrained weights
    checkpoint_path = Path('saved_models/candy.pth')
    if checkpoint_path.exists():
        print(f"✅ Found pre-trained weights")
    else:
        print(f"⚠️ No pre-trained weights (using random init)")
        checkpoint_path = None

else:
    checkpoint_path = None

In [ ]:
# Load Fast Style Transfer Model
if torch.cuda.is_available():
    from models.transformer_net import TransformerNet
    
    style_model = TransformerNet(num_residual_blocks=5).to(device)
    
    if checkpoint_path and checkpoint_path.exists():
        style_model.load_checkpoint(str(checkpoint_path))
        print("✅ Loaded pre-trained weights")
    
    style_model.eval()
    
    total_params = sum(p.numel() for p in style_model.parameters())
    print(f"Parameters: {total_params:,}")
    print(f"✅ Model loaded")

else:
    style_model = None

In [ ]:
# Test with random input
if torch.cuda.is_available() and style_model is not None:
    test_input = torch.randn(1, 3, 256, 256, device=device)
    
    with torch.no_grad():
        output = style_model(test_input)
    
    print(f"Input: {test_input.shape}")
    print(f"Output: {output.shape}")
    print("✅ Fast Style Transfer working!")

## 5. Image Upload & Style Transfer

Upload your own images to apply style transfer.

### Instructions:
1. Run the cell below
2. Click "Choose files" to upload an image
3. The stylized result will be displayed and available for download

In [ ]:
if torch.cuda.is_available() and style_model is not None:
    try:
        from google.colab import files
        from io import BytesIO
        from PIL import Image
        import matplotlib.pyplot as plt
        from torchvision import transforms
        
        print("=" * 70)
        print("Image Upload & Style Transfer")
        print("=" * 70)
        print("\n📁 Upload an image:\n")
        
        uploaded = files.upload()
        
        if uploaded:
            for filename in uploaded.keys():
                print(f"\nProcessing {filename}...")
                
                img = Image.open(BytesIO(uploaded[filename])).convert('RGB')
                original_size = img.size
                
                # Resize for processing
                PROCESSING_SIZE = 512
                aspect = img.size[0] / img.size[1]
                if aspect > 1:
                    new_size = (PROCESSING_SIZE, int(PROCESSING_SIZE / aspect))
                else:
                    new_size = (int(PROCESSING_SIZE * aspect), PROCESSING_SIZE)
                img_resized = img.resize(new_size, Image.Resampling.LANCZOS)
                
                # Convert to tensor
                transform = transforms.Compose([transforms.ToTensor()])
                input_tensor = transform(img_resized).unsqueeze(0).to(device)
                
                # Apply style transfer
                with torch.no_grad():
                    start = time.perf_counter()
                    output_tensor = style_model(input_tensor)
                    torch.cuda.synchronize()
                    elapsed_ms = (time.perf_counter() - start) * 1000
                
                # Convert back
                output_img = transforms.ToPILImage()(output_tensor.squeeze(0).clamp(0, 1))
                output_img = output_img.resize(original_size, Image.Resampling.LANCZOS)
                
                # Display
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                axes[0].imshow(img)
                axes[0].set_title('Original')
                axes[0].axis('off')
                axes[1].imshow(output_img)
                axes[1].set_title(f'Stylized ({elapsed_ms:.1f} ms)')
                axes[1].axis('off')
                plt.tight_layout()
                plt.show()
                
                # Save and download
                result_filename = f'stylized_{filename}'
                output_img.save(result_filename, quality=95)
                print(f"✅ Saved: {result_filename}")
                files.download(result_filename)
    
    except ImportError:
        print("\nNote: Image upload works in Google Colab.")
        print("For local usage, use PIL.Image.open()")

else:
    print("⚠️ CUDA not available or model not loaded")

## 6. ViT-Based Style Transfer

Vision Transformer-based style transfer using custom CUDA attention kernels.

### Model Variants:
| Variant | Parameters | Patches | Blocks |
|---------|------------|---------|--------|
| **nano** | 2M | 64 | 2 |
| **small** | 11M | 64 | 4 |
| **base** | 54M | 64 | 6 |

In [ ]:
if torch.cuda.is_available():
    from models.vit_style_transfer import create_model, STYLEFORGE_MODELS
    
    print("=" * 70)
    print("ViT Style Transfer Setup")
    print("=" * 70)
    
    print("\nAvailable variants:")
    for variant, config in STYLEFORGE_MODELS.items():
        print(f"  {variant}: {config['image_size']}, {config['embed_dim']} dim")
    
    # Create small model
    vit_model = create_model(variant='small', use_cuda_kernels=True).to(device)
    vit_model.eval()
    
    total_params = sum(p.numel() for p in vit_model.parameters())
    print(f"\nParameters: {total_params:,}")
    print("✅ ViT model loaded")
    
    vit_model_available = True

else:
    vit_model_available = False
    print("⚠️ CUDA not available")

In [ ]:
# Test ViT model
if torch.cuda.is_available() and vit_model_available:
    from models.vit_style_transfer import STYLEFORGE_MODELS
    
    config = STYLEFORGE_MODELS['small']
    IMAGE_SIZE = config['image_size']
    
    content = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
    style = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
    
    # Warmup
    with torch.no_grad():
        for _ in range(5):
            _ = vit_model(content, style)
    torch.cuda.synchronize()
    
    # Benchmark
    times = []
    with torch.no_grad():
        for _ in range(10):
            start = time.perf_counter()
            output = vit_model(content, style)
            torch.cuda.synchronize()
            times.append((time.perf_counter() - start) * 1000)
    
    avg_time = np.mean(times)
    fps = 1000 / avg_time
    
    print(f"\nAverage: {avg_time:.2f} ms")
    print(f"FPS: {fps:.2f}")
    print(f"Output: {output.shape}")
    print("\n✅ ViT Style Transfer working!")

else:
    print("⚠️ CUDA not available or ViT model not loaded")

## 7. TransformerNet Variant Comparison

Compare three implementations of the Johnson et al. architecture:

| Variant | Description | Speedup |
|---------|-------------|--------|
| **Baseline** | Pure PyTorch, no CUDA kernels | 1.0x |
| **Auto** | FusedInstanceNorm2d when available | 2-4x |
| **Fused** | Fully fused Conv+IN+ReLU (shared memory tiling) | 5-8x |

In [ ]:
print("=" * 70)
print("TransformerNet Variant Comparison")
print("=" * 70)

from models.transformer_net import (
    TransformerNet,
    TransformerNetBaseline,
    TransformerNetFused,
    get_available_variants,
)

print(f"\nAvailable variants: {', '.join(get_available_variants())}")

# Test size
TEST_SIZE = 512
x_test = torch.randn(1, 3, TEST_SIZE, TEST_SIZE, device=device)

variants = [
    ("baseline", TransformerNetBaseline),
    ("auto", TransformerNet),
    ("fused", TransformerNetFused),
]

results_variants = []

for variant_name, model_class in variants:
    try:
        print(f"\n{variant_name.upper()} - Creating model...", end="", flush=True)
        model = model_class(num_residual_blocks=5).to(device)
        model.eval()
        
        # Warmup
        with torch.no_grad():
            for _ in range(10):
                _ = model(x_test)
        torch.cuda.synchronize()
        
        # Benchmark
        times = []
        with torch.no_grad():
            for _ in range(30):
                start = torch.cuda.Event(enable_timing=True)
                end = torch.cuda.Event(enable_timing=True)
                start.record()
                _ = model(x_test)
                end.record()
                torch.cuda.synchronize()
                times.append(start.elapsed_time(end))
        
        avg_ms = np.mean(times)
        fps = 1000 / avg_ms
        
        results_variants.append({
            'variant': variant_name,
            'avg_ms': avg_ms,
            'fps': fps,
        })
        
        print(f"\r{variant_name.upper():10} {avg_ms:6.2f} ms  ({fps:5.1f} FPS)", flush=True)
        
    except Exception as e:
        print(f"\r{variant_name.upper():10} ERROR: {e}")

# Print comparison
if len(results_variants) >= 2:
    baseline_ms = results_variants[0]['avg_ms']
    print(f"\n{'='*50}")
    print("SPEEDUP VS BASELINE")
    print(f"{'='*50}")
    
    for r in results_variants[1:]:
        speedup = baseline_ms / r['avg_ms']
        print(f"{r['variant'].upper():10} {speedup:+.2f}x")

print(f"\n{'='*70}")

## 8. Individual Kernel Benchmarks

Benchmark each CUDA kernel independently against PyTorch baseline.

### 8.1 FusedInstanceNorm2d Benchmark

In [ ]:
print("=" * 70)
print("FusedInstanceNorm2d Benchmark")
print("=" * 70)

from kernels import FusedInstanceNorm2d

# Configs to test
norm_configs = [
    ("Small", 1, 64, 64, 64),
    ("Medium", 1, 128, 128, 128),
    ("Large", 1, 256, 256, 256),
]

print(f"\n{'Config':<12} {'PyTorch':<12} {'Fused':<12} {'Speedup':<10}")
print("-" * 50)

for name, b, c, h, w in norm_configs:
    x = torch.randn(b, c, h, w, device=device)
    
    # PyTorch baseline
    pytorch_norm = nn.InstanceNorm2d(c, affine=True).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = pytorch_norm(x)
    torch.cuda.synchronize()
    
    times_pytorch = []
    with torch.no_grad():
        for _ in range(50):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = pytorch_norm(x)
            end.record()
            torch.cuda.synchronize()
            times_pytorch.append(start.elapsed_time(end))
    
    # Fused kernel
    fused_norm = FusedInstanceNorm2d(c, affine=True).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = fused_norm(x)
    torch.cuda.synchronize()
    
    times_fused = []
    with torch.no_grad():
        for _ in range(50):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = fused_norm(x)
            end.record()
            torch.cuda.synchronize()
            times_fused.append(start.elapsed_time(end))
    
    avg_pytorch = np.mean(times_pytorch)
    avg_fused = np.mean(times_fused)
    speedup = avg_pytorch / avg_fused
    
    print(f"{name:<12} {avg_pytorch:8.2f} ms  {avg_fused:8.2f} ms  {speedup:6.2f}x")

print(f"\n{'='*70}")

### 8.2 FusedConvInstanceNormReLU Benchmark

This kernel uses **shared memory tiling** for K×K convolutions and **float4 vectorization** for 1×1 convolutions, achieving 5-8x speedup over PyTorch.

In [ ]:
print("=" * 70)
print("FusedConvInstanceNormReLU Benchmark")
print("=" * 70)

from kernels import FusedConvInstanceNormReLU

# Create PyTorch baseline: Conv2d + InstanceNorm2d + ReLU
class PyTorchConvINReLU(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride):
        super().__init__()
        self.pad = nn.ReflectionPad2d(kernel_size // 2)
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride)
        self.norm = nn.InstanceNorm2d(out_ch, affine=True)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.pad(x)
        x = self.conv(x)
        x = self.norm(x)
        return self.relu(x)

# Configs to test
conv_configs = [
    ("64ch", 1, 64, 64, 128, 128),
    ("128ch", 1, 128, 128, 128, 128),
]

print(f"\n{'Config':<12} {'PyTorch':<12} {'Fused':<12} {'Speedup':<10}")
print("-" * 50)

for name, b, c_in, h, w, c_out in conv_configs:
    x = torch.randn(b, c_in, h, w, device=device)
    
    # PyTorch baseline
    pytorch_layer = PyTorchConvINReLU(c_in, c_out, 3, 1).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = pytorch_layer(x)
    torch.cuda.synchronize()
    
    times_pytorch = []
    with torch.no_grad():
        for _ in range(50):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = pytorch_layer(x)
            end.record()
            torch.cuda.synchronize()
            times_pytorch.append(start.elapsed_time(end))
    
    # Fused kernel
    fused_layer = FusedConvInstanceNormReLU(c_in, c_out, 3, 1).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = fused_layer(x)
    torch.cuda.synchronize()
    
    times_fused = []
    with torch.no_grad():
        for _ in range(50):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = fused_layer(x)
            end.record()
            torch.cuda.synchronize()
            times_fused.append(start.elapsed_time(end))
    
    avg_pytorch = np.mean(times_pytorch)
    avg_fused = np.mean(times_fused)
    speedup = avg_pytorch / avg_fused
    
    print(f"{name:<12} {avg_pytorch:8.2f} ms  {avg_fused:8.2f} ms  {speedup:6.2f}x")

print(f"\n{'='*70}")

### 8.3 FusedAttentionV3 Benchmark

In [ ]:
print("=" * 70)
print("FusedAttentionV3 Benchmark")
print("=" * 70)

from kernels import FusedAttentionV3

# Configs to test
attn_configs = [
    ("Small", 2, 64, 128, 4),
    ("Medium", 2, 128, 256, 8),
    ("Large", 2, 256, 512, 16),
]

print(f"\n{'Config':<12} {'PyTorch':<12} {'Fused':<12} {'Speedup':<10}")
print("-" * 50)

for name, b, seq_len, embed_dim, num_heads in attn_configs:
    q = torch.randn(b, seq_len, embed_dim, device=device)
    k = torch.randn(b, seq_len, embed_dim, device=device)
    v = torch.randn(b, seq_len, embed_dim, device=device)
    
    # PyTorch baseline (naive multi-head attention)
    class PyTorchAttention(nn.Module):
        def __init__(self, embed_dim, num_heads):
            super().__init__()
            self.embed_dim = embed_dim
            self.num_heads = num_heads
            self.head_dim = embed_dim // num_heads
            self.qkv = nn.Linear(embed_dim, 3 * embed_dim)
            self.out = nn.Linear(embed_dim, embed_dim)
        
        def forward(self, q, k, v):
            B, L, D = q.shape
            qkv = self.qkv(torch.stack([q, k, v], dim=0).permute(1,0,2))
            qkv = qkv.reshape(B, 3, self.num_heads, self.head_dim, L).permute(1,3,0,2,4)
            q, k, v = qkv[0], qkv[1], qkv[2]
            scale = self.head_dim ** -0.5
            attn = (q @ k.transpose(-2,-1)) * scale
            attn = attn.softmax(dim=-1)
            out = (attn @ v).transpose(1,2).reshape(B, L, D)
            return self.out(out)
    
    pytorch_attn = PyTorchAttention(embed_dim, num_heads).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = pytorch_attn(q, k, v)
    torch.cuda.synchronize()
    
    times_pytorch = []
    with torch.no_grad():
        for _ in range(30):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = pytorch_attn(q, k, v)
            end.record()
            torch.cuda.synchronize()
            times_pytorch.append(start.elapsed_time(end))
    
    # Fused kernel
    fused_attn = FusedAttentionV3(embed_dim=embed_dim, num_heads=num_heads).to(device).eval()
    with torch.no_grad():
        for _ in range(10):
            _ = fused_attn(q, k, v)
    torch.cuda.synchronize()
    
    times_fused = []
    with torch.no_grad():
        for _ in range(30):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            _ = fused_attn(q, k, v)
            end.record()
            torch.cuda.synchronize()
            times_fused.append(start.elapsed_time(end))
    
    avg_pytorch = np.mean(times_pytorch)
    avg_fused = np.mean(times_fused)
    speedup = avg_pytorch / avg_fused
    
    print(f"{name:<12} {avg_pytorch:8.2f} ms  {avg_fused:8.2f} ms  {speedup:6.2f}x")

print(f"\n{'='*70}")

## 9. Summary & Achievements

### CUDA Kernels Implemented

| Kernel | Purpose | Optimization | Speedup | Status |
|--------|---------|--------------|--------|--------|
| FusedInstanceNorm2d | Fused normalization | Warp reductions, single kernel | 2-4x | ✅ Production-ready |
| FusedConvInstanceNormReLU | Conv+IN+ReLU fused | Shared memory tiling, float4 vectorization | 5-8x | ✅ Production-ready |
| FusedAttentionV3 | Multi-head attention | Vectorized memory access | 4-8x | ✅ Working |

### TransformerNet Variants

| Variant | Kernel | Speedup | Use Case |
|---------|--------|--------|----------|
| Baseline | None | 1.0x | CPU, debugging |
| Auto | FusedInstanceNorm2d | 2-4x | General use |
| Fused | FusedConv+IN+ReLU | 5-8x | Real-time applications |

### Key Optimizations in FusedConvInstanceNormReLU

1. **Shared Memory Tiling**: Reduces global memory traffic by ~K² factor
   - Each thread block cooperatively loads input tile into shared memory
   - Threads reuse shared data for kernel computation
   - Eliminates redundant global memory reads

2. **Vectorized 1×1 Convolution**: Uses float4 for 4× memory bandwidth
   - Processes 4 channels per iteration
   - Critical for residual blocks with 1×1 bottlenecks

3. **Coalesced Memory Access**: Threads access consecutive memory locations
   - Maximizes memory bus utilization
   - Reduces memory transaction count

### How to Use

```python
# Import kernels
from kernels import FusedInstanceNorm2d, FusedConvInstanceNormReLU, FusedAttentionV3

# Import models
from models.transformer_net import TransformerNet, TransformerNetFused, create_transformer_net

# Use fused normalization
norm = FusedInstanceNorm2d(64).cuda()
x = torch.randn(1, 64, 256, 256).cuda()
y = norm(x)

# Use fused conv layer
conv = FusedConvInstanceNormReLU(64, 128, 3).cuda()
y = conv(x)

# Use variant model
model = create_transformer_net(variant='fused')
```

### Running Benchmarks

```bash
# Variant comparison
python benchmark_style_transfer_variants.py

# Full benchmark suite
python run_full_benchmark.py
```